In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from PIL import Image
import cv2
import torch
from torchvision import transforms, datasets
import numpy as np
import os

In [3]:
!unzip /content/drive/MyDrive/GAF_3.0.zip

Archive:  /content/drive/MyDrive/GAF_3.0.zip
 extracting: Train/Negative.zip      
 extracting: Train/Neutral.zip       
 extracting: Train/Positive.zip      
 extracting: Validation/Negative.zip  
 extracting: Validation/Neutral.zip  
 extracting: Validation/Positive.zip  
 extracting: ReadMe.txt              
 extracting: Test_Images.zip         


In [5]:
!unzip /content/mtcnn/CroppedFaces.zip

Streaming output truncated to the last 5000 lines.
  inflating: CroppedFaces/train/Positive/pos_413.npz  
  inflating: CroppedFaces/train/Positive/pos_414.npz  
  inflating: CroppedFaces/train/Positive/pos_415.npz  
  inflating: CroppedFaces/train/Positive/pos_416.npz  
  inflating: CroppedFaces/train/Positive/pos_417.npz  
  inflating: CroppedFaces/train/Positive/pos_418.npz  
  inflating: CroppedFaces/train/Positive/pos_419.npz  
  inflating: CroppedFaces/train/Positive/pos_42.npz  
  inflating: CroppedFaces/train/Positive/pos_420.npz  
  inflating: CroppedFaces/train/Positive/pos_421.npz  
  inflating: CroppedFaces/train/Positive/pos_422.npz  
  inflating: CroppedFaces/train/Positive/pos_423.npz  
  inflating: CroppedFaces/train/Positive/pos_424.npz  
  inflating: CroppedFaces/train/Positive/pos_425.npz  
  inflating: CroppedFaces/train/Positive/pos_426.npz  
  inflating: CroppedFaces/train/Positive/pos_427.npz  
  inflating: CroppedFaces/train/Positive/pos_428.npz  
  inflating: Cr

In [4]:
!unzip /content/drive/MyDrive/mtcnn.zip

Archive:  /content/drive/MyDrive/mtcnn.zip
 extracting: mtcnn/AlignedCroppedImages.zip  
 extracting: mtcnn/CroppedFaces.zip  
  inflating: mtcnn/FaceCoordinates.rar  
 extracting: mtcnn/FaceFeatures.zip  
  inflating: mtcnn/test_list         
  inflating: mtcnn/train_annotations.npz  
  inflating: mtcnn/val_annotations.npz  
  inflating: mtcnn/val_list          


In [6]:
neg_train = sorted(os.listdir('/content/CroppedFaces/train/Negative/')) #/content/CroppedFaces/train/Negative
neu_train = sorted(os.listdir('/content/CroppedFaces/train/Neutral/'))
pos_train = sorted(os.listdir('/content/CroppedFaces/train/Positive/'))

print(len(neg_train))
print(len(neu_train))
print(len(pos_train))

2758
3080
3977


In [7]:
neg_val = sorted(os.listdir('/content/CroppedFaces/val/Negative/')) #/content/CroppedFaces/val/Negative
neu_val = sorted(os.listdir('/content/CroppedFaces/val/Neutral/'))
pos_val = sorted(os.listdir('/content/CroppedFaces/val/Positive/'))

print(len(neg_val))
print(len(neu_val))
print(len(pos_val))

1231
1368
1747


In [8]:
!unzip /content/mtcnn/AlignedCroppedImages.zip

Streaming output truncated to the last 5000 lines.
  inflating: AlignedCroppedImages/val/Positive/pos_332_0.jpg  
  inflating: AlignedCroppedImages/val/Positive/pos_332_1.jpg  
  inflating: AlignedCroppedImages/val/Positive/pos_332_2.jpg  
  inflating: AlignedCroppedImages/val/Positive/pos_332_3.jpg  
  inflating: AlignedCroppedImages/val/Positive/pos_332_4.jpg  
  inflating: AlignedCroppedImages/val/Positive/pos_332_5.jpg  
  inflating: AlignedCroppedImages/val/Positive/pos_333_0.jpg  
  inflating: AlignedCroppedImages/val/Positive/pos_333_1.jpg  
  inflating: AlignedCroppedImages/val/Positive/pos_334_0.jpg  
  inflating: AlignedCroppedImages/val/Positive/pos_334_1.jpg  
  inflating: AlignedCroppedImages/val/Positive/pos_334_2.jpg  
  inflating: AlignedCroppedImages/val/Positive/pos_334_3.jpg  
  inflating: AlignedCroppedImages/val/Positive/pos_334_4.jpg  
  inflating: AlignedCroppedImages/val/Positive/pos_334_5.jpg  
  inflating: AlignedCroppedImages/val/Positive/pos_334_6.jpg  
  in

In [10]:
for j in range(len(neg_train)):
    j = j+1
    print(j)
    data = np.load("/content/CroppedFaces/train/Negative/neg_"+str(j)+".npz", allow_pickle=True) #/content/CroppedFaces/train/Negative
    
    faces = data['a']
    landmarks = data['b']
    desiredFaceWidth = 112
    desiredFaceHeight = 96
    desiredLeftEye = [0.30, 0.30]
    desiredRightEyeX = 1.0 - desiredLeftEye[0]
    
    if landmarks.all() == [0]:
        continue
            
    for i in range(len(faces)):
        leftEyeCenter = [landmarks[i][0], landmarks[i][5]]
        rightEyeCenter = [landmarks[i][1], landmarks[i][6]]

        dY = rightEyeCenter[1] - leftEyeCenter[1]
        dX = rightEyeCenter[0] - leftEyeCenter[0]
        angle = np.degrees(np.arctan2(dY, dX))

        eyesCenter = ((leftEyeCenter[0]+rightEyeCenter[0])/2,(leftEyeCenter[1]+rightEyeCenter[1])/2)

        dist = np.sqrt((dX ** 2) + (dY ** 2))
        desiredDist = (desiredRightEyeX - desiredLeftEye[0])
        desiredDist *= desiredFaceWidth
        scale = desiredDist / dist

        M = cv2.getRotationMatrix2D(eyesCenter, angle, scale)

        tX = desiredFaceWidth * 0.5
        tY = desiredFaceHeight * desiredLeftEye[1]
        M[0, 2] += (tX - eyesCenter[0])
        M[1, 2] += (tY - eyesCenter[1])

        (w, h) = (desiredFaceWidth, desiredFaceHeight)
        output = cv2.warpAffine(faces[i], M, (w, h),flags=cv2.INTER_CUBIC)
        
        im = Image.fromarray(output)
        im.save(r"\content\AlignedCroppedImages\train\Negative\neg_"+str(j)+"_"+str(i)+".jpg") #/content/AlignedCroppedImages/train/Negative

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [12]:
for j in range(len(neu_train)):
    j = j+1
    print(j)
    data = np.load("/content/CroppedFaces/train/Neutral/neu_"+str(j)+".npz", allow_pickle=True) #/content/CroppedFaces/train/Negative
    
    faces = data['a']
    landmarks = data['b']
    desiredFaceWidth = 112
    desiredFaceHeight = 96
    desiredLeftEye = [0.30, 0.30]
    desiredRightEyeX = 1.0 - desiredLeftEye[0]
    
    if landmarks.all() == [0]:
        continue
            
    for i in range(len(faces)):
        leftEyeCenter = [landmarks[i][0], landmarks[i][5]]
        rightEyeCenter = [landmarks[i][1], landmarks[i][6]]

        dY = rightEyeCenter[1] - leftEyeCenter[1]
        dX = rightEyeCenter[0] - leftEyeCenter[0]
        angle = np.degrees(np.arctan2(dY, dX))

        eyesCenter = ((leftEyeCenter[0]+rightEyeCenter[0])/2,(leftEyeCenter[1]+rightEyeCenter[1])/2)

        dist = np.sqrt((dX ** 2) + (dY ** 2))
        desiredDist = (desiredRightEyeX - desiredLeftEye[0])
        desiredDist *= desiredFaceWidth
        scale = desiredDist / dist

        M = cv2.getRotationMatrix2D(eyesCenter, angle, scale)

        tX = desiredFaceWidth * 0.5
        tY = desiredFaceHeight * desiredLeftEye[1]
        M[0, 2] += (tX - eyesCenter[0])
        M[1, 2] += (tY - eyesCenter[1])

        (w, h) = (desiredFaceWidth, desiredFaceHeight)
        output = cv2.warpAffine(faces[i], M, (w, h),flags=cv2.INTER_CUBIC)
        
        im = Image.fromarray(output)
        im.save(r"\content\AlignedCroppedImages\train\Neutral\neu_"+str(j)+"_"+str(i)+".jpg")

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [13]:
for j in range(len(pos_train)):
    j = j+1
    print(j)
    data = np.load("/content/CroppedFaces/train/Positive/pos_"+str(j)+".npz",allow_pickle=True) #/content/CroppedFaces/train/Negative
    
    faces = data['a']
    landmarks = data['b']
    desiredFaceWidth = 112
    desiredFaceHeight = 96
    desiredLeftEye = [0.30, 0.30]
    desiredRightEyeX = 1.0 - desiredLeftEye[0]
    
    if landmarks.all() == [0]:
        continue
            
    for i in range(len(faces)):
        leftEyeCenter = [landmarks[i][0], landmarks[i][5]]
        rightEyeCenter = [landmarks[i][1], landmarks[i][6]]

        dY = rightEyeCenter[1] - leftEyeCenter[1]
        dX = rightEyeCenter[0] - leftEyeCenter[0]
        angle = np.degrees(np.arctan2(dY, dX))

        eyesCenter = ((leftEyeCenter[0]+rightEyeCenter[0])/2,(leftEyeCenter[1]+rightEyeCenter[1])/2)

        dist = np.sqrt((dX ** 2) + (dY ** 2))
        desiredDist = (desiredRightEyeX - desiredLeftEye[0])
        desiredDist *= desiredFaceWidth
        scale = desiredDist / dist

        M = cv2.getRotationMatrix2D(eyesCenter, angle, scale)

        tX = desiredFaceWidth * 0.5
        tY = desiredFaceHeight * desiredLeftEye[1]
        M[0, 2] += (tX - eyesCenter[0])
        M[1, 2] += (tY - eyesCenter[1])

        (w, h) = (desiredFaceWidth, desiredFaceHeight)
        output = cv2.warpAffine(faces[i], M, (w, h),flags=cv2.INTER_CUBIC)
        
        im = Image.fromarray(output)
        im.save(r"\content\AlignedCroppedImages\train\Positive\pos_"+str(j)+"_"+str(i)+".jpg")

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [14]:
for j in range(len(neg_val)):
    j = j+1
    print(j)
    data = np.load("/content/CroppedFaces/val/Negative/neg_"+str(j)+".npz", allow_pickle=True)
    
    faces = data['a']
    landmarks = data['b']
    desiredFaceWidth = 112
    desiredFaceHeight = 96
    desiredLeftEye = [0.30, 0.30]
    desiredRightEyeX = 1.0 - desiredLeftEye[0]
    
    if landmarks.all() == [0]:
        continue
            
    for i in range(len(faces)):
        leftEyeCenter = [landmarks[i][0], landmarks[i][5]]
        rightEyeCenter = [landmarks[i][1], landmarks[i][6]]

        dY = rightEyeCenter[1] - leftEyeCenter[1]
        dX = rightEyeCenter[0] - leftEyeCenter[0]
        angle = np.degrees(np.arctan2(dY, dX))

        eyesCenter = ((leftEyeCenter[0]+rightEyeCenter[0])/2,(leftEyeCenter[1]+rightEyeCenter[1])/2)

        dist = np.sqrt((dX ** 2) + (dY ** 2))
        desiredDist = (desiredRightEyeX - desiredLeftEye[0])
        desiredDist *= desiredFaceWidth
        scale = desiredDist / dist

        M = cv2.getRotationMatrix2D(eyesCenter, angle, scale)

        tX = desiredFaceWidth * 0.5
        tY = desiredFaceHeight * desiredLeftEye[1]
        M[0, 2] += (tX - eyesCenter[0])
        M[1, 2] += (tY - eyesCenter[1])

        (w, h) = (desiredFaceWidth, desiredFaceHeight)
        output = cv2.warpAffine(faces[i], M, (w, h),flags=cv2.INTER_CUBIC)
        
        im = Image.fromarray(output)
        im.save(r"\content\AlignedCroppedImages\val\Negative\neg_"+str(j)+"_"+str(i)+".jpg")

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [15]:
for j in range(len(neu_val)):
    j = j+1
    print(j)
    data = np.load("/content/CroppedFaces/val/Neutral/neu_"+str(j)+".npz", allow_pickle=True)
    
    faces = data['a']
    landmarks = data['b']
    desiredFaceWidth = 112
    desiredFaceHeight = 96
    desiredLeftEye = [0.30, 0.30]
    desiredRightEyeX = 1.0 - desiredLeftEye[0]
    
    if landmarks.all() == [0]:
        continue
            
    for i in range(len(faces)):
        leftEyeCenter = [landmarks[i][0], landmarks[i][5]]
        rightEyeCenter = [landmarks[i][1], landmarks[i][6]]

        dY = rightEyeCenter[1] - leftEyeCenter[1]
        dX = rightEyeCenter[0] - leftEyeCenter[0]
        angle = np.degrees(np.arctan2(dY, dX))

        eyesCenter = ((leftEyeCenter[0]+rightEyeCenter[0])/2,(leftEyeCenter[1]+rightEyeCenter[1])/2)

        dist = np.sqrt((dX ** 2) + (dY ** 2))
        desiredDist = (desiredRightEyeX - desiredLeftEye[0])
        desiredDist *= desiredFaceWidth
        scale = desiredDist / dist

        M = cv2.getRotationMatrix2D(eyesCenter, angle, scale)

        tX = desiredFaceWidth * 0.5
        tY = desiredFaceHeight * desiredLeftEye[1]
        M[0, 2] += (tX - eyesCenter[0])
        M[1, 2] += (tY - eyesCenter[1])

        (w, h) = (desiredFaceWidth, desiredFaceHeight)
        output = cv2.warpAffine(faces[i], M, (w, h),flags=cv2.INTER_CUBIC)
        
        im = Image.fromarray(output)
        im.save(r"\content\AlignedCroppedImages\val\Neutral\neu_"+str(j)+"_"+str(i)+".jpg")

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [16]:
for j in range(len(pos_val)):
    j = j+1
    print(j)
    data = np.load("/content/CroppedFaces/val/Positive/pos_"+str(j)+".npz", allow_pickle=True)
    
    faces = data['a']
    landmarks = data['b']
    desiredFaceWidth = 112
    desiredFaceHeight = 96
    desiredLeftEye = [0.30, 0.30]
    desiredRightEyeX = 1.0 - desiredLeftEye[0]
    
    if landmarks.all() == [0]:
        continue
            
    for i in range(len(faces)):
        leftEyeCenter = [landmarks[i][0], landmarks[i][5]]
        rightEyeCenter = [landmarks[i][1], landmarks[i][6]]

        dY = rightEyeCenter[1] - leftEyeCenter[1]
        dX = rightEyeCenter[0] - leftEyeCenter[0]
        angle = np.degrees(np.arctan2(dY, dX))

        eyesCenter = ((leftEyeCenter[0]+rightEyeCenter[0])/2,(leftEyeCenter[1]+rightEyeCenter[1])/2)

        dist = np.sqrt((dX ** 2) + (dY ** 2))
        desiredDist = (desiredRightEyeX - desiredLeftEye[0])
        desiredDist *= desiredFaceWidth
        scale = desiredDist / dist

        M = cv2.getRotationMatrix2D(eyesCenter, angle, scale)

        tX = desiredFaceWidth * 0.5
        tY = desiredFaceHeight * desiredLeftEye[1]
        M[0, 2] += (tX - eyesCenter[0])
        M[1, 2] += (tY - eyesCenter[1])

        (w, h) = (desiredFaceWidth, desiredFaceHeight)
        output = cv2.warpAffine(faces[i], M, (w, h),flags=cv2.INTER_CUBIC)
        
        im = Image.fromarray(output)
        im.save(r"\content\AlignedCroppedImages\val\Positive\pos_"+str(j)+"_"+str(i)+".jpg")

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
